In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 10
height = 10
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 600

puzzle.naming = {
    None: '  ',
    'n': '  ',
    'e': ' _',
    'u': ' ^',
    'd': ' v',
    'l': ' <',
    'r': ' >',
    'o': ' o',
    's': ' .',
    'sub(u)': '*^',
    'sub(d)': '*v',
    'sub(l)': '*<',
    'sub(r)': '*>',
    'sub(o)': '*o',
    'sub(s)': '*.',
}
for i in range(max(width, height)):
    puzzle.naming[str(i)] = f" {i}"

puzzle.domain_program += """
    value(e). % empty sea
    value(n). % null value
    ship_value(u;d;l;r). % ends of long ships
    ship_value(o). % middle part of long ships
    ship_value(s). % short ships
    value(S) :- ship_value(S).
    sub_value(sub(S)) :- ship_value(S).
    value(S) :- sub_value(S).
    num(0..board_width-1).
    num(0..board_height-1).
    value(N) :- num(N).
    
    main_cell(c(Row,Col)) :- cell(c(Row,Col)),
        Row < board_height, Col < board_width.
    bottom_counting_cell(c(Row,Col)) :- cell(c(Row,Col)),
        Row = board_height, Col < board_width.
    right_counting_cell(c(Row,Col)) :- cell(c(Row,Col)),
        Row < board_height, Col = board_width.
    empty_cell(C) :- cell(C),
        not main_cell(C),
        not bottom_counting_cell(C),
        not right_counting_cell(C).
    
    ship_size(1..4,1).
    ship_size(5..7,2).
    ship_size(8..9,3).
    submarine(8).
    ship_size(10,4).
    ship(S) :- ship_size(S,_).
    ship_size(N) :- ship_size(_,N).
    long_ship(S) :- ship(S), ship_size(S,N), N > 1.
    short_ship(S) :- ship(S), not long_ship(S).
    
    guessed_number(num_ships_of_length(N),M) :-
        ship_size(N),
        M = #count { S : ship_size(S,N) }.
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) } :- main_cell(C), solution(C,V), not sub_value(V).
    { puzzle(C,e) } :- main_cell(C), solution(C,V), sub_value(V).
    puzzle(C,V) :- solution(C,V), cell(C), not main_cell(C).
"""
puzzle.puzzle_constraints_program += """
    #minimize { 1,puzzle(C,V) : puzzle(C,V) }.
    
    :- puzzle(C,s).
    :- adjacent_cells(C1,C2),
        puzzle(C1,V1), ship_value(V1),
        puzzle(C2,V2), ship_value(V2).
    :- adjacent_cells(C1,C2), puzzle(C1,V1), ship_value(V1), puzzle(C2,e).
    %:- adjacent_cells(C1,C2), puzzle(C1,e), puzzle(C2,e).
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V), V != e.
    :- puzzle(C,e), solution(C,V), ship_value(V).
"""
puzzle.solution_program += """
    :- main_cell(C), solution(C,V), num(V).
    :- empty_cell(C), solution(C,V), V != n.
    :- not empty_cell(C), solution(C,n).
    :- bottom_counting_cell(C), solution(C,V), not num(V).
    :- right_counting_cell(C), solution(C,V), not num(V).
    
    same_cells(C,C) :- cell(C).
    adjacent_cells(c(R1,C1),c(R2,C2)) :-
        cell(c(R1,C1)), cell(c(R2,C2)),
        |R1-R2| <= 1, |C1-C2| <= 1, not same_cells(c(R1,C1),c(R2,C2)).
    
    :- right_counting_cell(C), C = c(Row,Col1),
        N = #count { C2 : C2 = c(Row,Col2), main_cell(C2), solution(C2,V), ship_value(V) },
        not solution(C,N).
    :- bottom_counting_cell(C), C = c(Row1,Col),
        N = #count { C2 : C2 = c(Row2,Col), main_cell(C2), solution(C2,V), ship_value(V) },
        not solution(C,N).
    
    1 { topleft(S,C) : cell(C) } 1 :- ship(S), not submarine(S).
    
    assigned(C,S) :- ship(S), topleft(S,C).
    
    1 { orientation(S,(h;v)) } 1 :- long_ship(S).
    
    :- short_ship(S), not submarine(S), topleft(S,C), not solution(C,s).
    :- short_ship(S), submarine(S), topleft(S,C), not solution(C,sub(s)).
    
    :- long_ship(S), topleft(S,C), orientation(S,h),
        ship_size(S,N), C = c(Row,Col),
        not main_cell(c(Row,Col+N-1)).
    assigned(C2,S) :- long_ship(S), topleft(S,C1), orientation(S,h),
        ship_size(S,N), C1 = c(Row,Col), C2 = c(Row,Col+N-1).
    assigned(C2,S) :- long_ship(S), topleft(S,C1), orientation(S,h),
        ship_size(S,N), C1 = c(Row,Col), C2 = c(Row,Col2),
        main_cell(C2), Col < Col2, Col2 < Col+N-1.
    
    :- long_ship(S), not submarine(S), topleft(S,C), orientation(S,h),
        not solution(C,l).
    :- long_ship(S), not submarine(S), topleft(S,C1), orientation(S,h),
        ship_size(S,N), C1 = c(Row,Col), C2 = c(Row,Col2),
        main_cell(C2), Col < Col2, Col2 < Col+N-1,
        not solution(C2,o).
    :- long_ship(S), not submarine(S), topleft(S,C), orientation(S,h),
        ship_size(S,N), C = c(Row,Col),
        not solution(c(Row,Col+N-1),r).
    
    :- long_ship(S), submarine(S), topleft(S,C), orientation(S,h),
        not solution(C,sub(l)).
    :- long_ship(S), submarine(S), topleft(S,C1), orientation(S,h),
        ship_size(S,N), C1 = c(Row,Col), C2 = c(Row,Col2),
        main_cell(C2), Col < Col2, Col2 < Col+N-1,
        not solution(C2,sub(o)).
    :- long_ship(S), submarine(S), topleft(S,C), orientation(S,h),
        ship_size(S,N), C = c(Row,Col),
        not solution(c(Row,Col+N-1),sub(r)).
    
    :- long_ship(S), topleft(S,C), orientation(S,v),
        ship_size(S,N), C = c(Row,Col),
        not main_cell(c(Row+N-1,Col)).
    assigned(C2,S) :- long_ship(S), topleft(S,C1), orientation(S,v),
        ship_size(S,N), C1 = c(Row,Col), C2 = c(Row+N-1,Col).
    assigned(C2,S) :- long_ship(S), topleft(S,C1), orientation(S,v),
        ship_size(S,N), C1 = c(Row,Col), C2 = c(Row2,Col),
        main_cell(C2), Row < Row2, Row2 < Row+N-1.
    
    :- long_ship(S), not submarine(S), topleft(S,C), orientation(S,v),
        not solution(C,u).
    :- long_ship(S), not submarine(S), topleft(S,C1), orientation(S,v),
        ship_size(S,N), C1 = c(Row,Col), C2 = c(Row2,Col),
        main_cell(C2), Row < Row2, Row2 < Row+N-1,
        not solution(C2,o).
    :- long_ship(S), not submarine(S), topleft(S,C), orientation(S,v),
        ship_size(S,N), C = c(Row,Col),
        not solution(c(Row+N-1,Col),d).
    
    :- long_ship(S), submarine(S), topleft(S,C), orientation(S,v),
        not solution(C,sub(u)).
    :- long_ship(S), submarine(S), topleft(S,C1), orientation(S,v),
        ship_size(S,N), C1 = c(Row,Col), C2 = c(Row2,Col),
        main_cell(C2), Row < Row2, Row2 < Row+N-1,
        not solution(C2,sub(o)).
    :- long_ship(S), submarine(S), topleft(S,C), orientation(S,v),
        ship_size(S,N), C = c(Row,Col),
        not solution(c(Row+N-1,Col),sub(d)).
    
    :- main_cell(C), not assigned(C,S) : ship(S); not solution(C,e).
    
    :- adjacent_cells(C1,C2),
        assigned(C1,S1), assigned(C2,S2), ship(S1), ship(S2), S1 != S2.
    :- same_cells(C1,C2),
        assigned(C1,S1), assigned(C2,S2), ship(S1), ship(S2), S1 != S2.

    :- main_cell(C), solution(C,V), ship_value(V),
        not assigned(C,S) : ship(S).
    
    %1 { topleft(S,C) : cell(C) } 1 :- ship(S), submarine(S).
"""

puzzle.designated_solution_constraints = [
    """
    1 { topleft(S,C) : cell(C) } 1 :- ship(S), submarine(S).
    """,
]

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

..................
Stopped after solving timeout..
Solving time: 620.69 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

|  |  |  |  |  |  |  |  |  | 1|
|  |  |  |  |  |  |  |  |  | 1|
|  | o|  | >|  |  |  |  |  | 4|
|  |  |  |  |  |  |  |  |  | 1|
|  |  | <|  |  |  |  |  | v| 4|
|  |  |  |  |  |  |  |  |  | 1|
| _|  |  |  |  | v|  |  |  | 1|
|  |  |  |  |  |  |  |  | >| 3|
|  |  |  |  |  |  |  |  |  | 1|
| 2| 1| 2| 4| 0| 3| 1| 1| 3|  |

num_ships_of_length(4) = 1
num_ships_of_length(1) = 4
num_ships_of_length(2) = 3
num_ships_of_length(3) = 2


In [5]:
print(puzzle.pretty_repr_solution())

| _| _| _| .| _| _| _| _| _| 1|
| _| _| _| _| _| _| .| _| _| 1|
| <| o| o| >| _| _| _| _| _| 4|
| _| _| _| _| _| _| _| _| ^| 1|
|*^| _| <| >| _| ^| _| _| v| 4|
|*o| _| _| _| _| o| _| _| _| 1|
|*v| _| _| _| _| v| _| _| _| 1|
| _| _| _| .| _| _| _| <| >| 3|
| .| _| _| _| _| _| _| _| _| 1|
| 2| 1| 2| 4| 0| 3| 1| 1| 3|  |
